# Explaibability Agent

Explaibability Agent (Creates the explainable output of the recommendation)

## **1. Load And Preprocess Data**

This part's only purpose is to load the data used in the Explainability Agent. This process is described in detail in the Preparation Agent. 

**Note: When computing the script with another Household than Household 1 you might need to adapt some parameters**

### **1.1 Initialize And Load Python Scripts**

In [1]:
import pandas as pd
import numpy as np
import os
import sqlite3
dir = 'D:/Master BWL HU/3. Semester/Seminar Information Systems/Seminar-Information-Systems-main'
os.chdir(dir)

from helper_functions import Helper
from agents import Preparation_Agent, Activity_Agent, Usage_Agent, Price_Agent, Load_Agent
import pandas as pd

helper = Helper()

dbfile  = "D:/Master BWL HU/3. Semester/Seminar Information Systems/Seminar-Information-Systems-main/home-assistant_Chris_v3.db"


C:\Users\chris\anaconda3\envs\recomm\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


### **1.2 Set Parameters For Pre-processing Step**

In [2]:
shiftable_devices = ["sensor.shellyplug_s_4022d88961b4_power", "sensor.shellyplug_s_4022d88984b8_power"]
date = '2023-01-09'
model_type = 'random forest'


truncation_params = {
    'features': 'all', 
    'factor': 1.5, 
    'verbose': 0
}

scale_params = {
    'features': 'all', 
    'kind': 'MinMax', 
    'verbose': 0
}

aggregate_params = {
    'resample_param': '60T'
}
aggregate_params24_H = {
    'resample_param': '24H'
}


activity_params = {
    'active_appliances': shiftable_devices,
    'threshold': .10
}

time_params = {
    'features': ['hour', 'day_name']
}

activity_lag_params = {
    'features': ['activity'],
    'lags': [24, 48, 72]
}

device = {
    'threshold' : .10}

activity_pipe_params = {
    'truncate': truncation_params,
    'scale': scale_params,
    'aggregate': aggregate_params,
    'activity': activity_params,
    'time': time_params,
    'activity_lag': activity_lag_params
}

usage_pipe_params = {
    'truncate': truncation_params,
    'scale': scale_params,
    'activity': activity_params,
    'aggregate_hour': aggregate_params,
    'aggregate_day': aggregate_params24_H,
    'time': time_params,
    'activity_lag': activity_lag_params,
    'shiftable_devices' : shiftable_devices,
    'device': device
}

device_params = {
    'threshold': 0.10
}

load_pipe_params = {
    'truncate': truncation_params,
    'scale': scale_params,
    'aggregate': aggregate_params,
    'shiftable_devices': shiftable_devices, 
    'device': device_params
}

### **1.3 Pre-process Data For Input In Device_Usage Agent**

In [3]:
device = shiftable_devices[1]

In [5]:
# calling the preparation pipeline
prep = Preparation_Agent(dbfile, shiftable_devices)

In [6]:
# creating activity df
activity_df = prep.pipeline_activity(prep.input, activity_pipe_params)
activity_df

,activity,hour,activity_lag_24,activity_lag_48,activity_lag_72,day_name_Monday,day_name_Saturday,day_name_Sunday,day_name_Thursday,day_name_Tuesday,day_name_Wednesday
last_updated,,,,,,,,,,,
2022-12-25 18:00:00,1,18,NaN,NaN,NaN,0,0,1,0,0,0
2022-12-25 19:00:00,0,19,NaN,NaN,NaN,0,0,1,0,0,0
2022-12-25 20:00:00,0,20,NaN,NaN,NaN,0,0,1,0,0,0
2022-12-25 21:00:00,0,21,NaN,NaN,NaN,0,0,1,0,0,0
2022-12-25 22:00:00,0,22,NaN,NaN,NaN,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
2023-01-10 19:00:00,0,19,0.0,0.0,0.0,0,0,0,0,1,0
2023-01-10 20:00:00,0,20,0.0,0.0,0.0,0,0,0,0,1,0
2023-01-10 21:00:00,0,21,0.0,0.0,0.0,0,0,0,0,1,0


In [7]:
activity = Activity_Agent(activity_df)

date = '2023-01-09'
split_params = {'train_start': '', 'test_delta': {'days':1, 'seconds':-1}, 'target': 'activity'}

activity_probs, X_train_activity, X_test_activity, model_activity = activity.pipeline_xai(activity_df, date, 'random forest', split_params)

In [8]:
activity_probs

last_updated
2023-01-10 00:00:00    0.000000
2023-01-10 01:00:00    0.000000
2023-01-10 02:00:00    0.000000
2023-01-10 03:00:00    0.000000
2023-01-10 04:00:00    0.000000
2023-01-10 05:00:00    0.000000
2023-01-10 06:00:00    0.000000
2023-01-10 07:00:00    0.018683
2023-01-10 08:00:00    0.027424
2023-01-10 09:00:00    0.009704
2023-01-10 10:00:00    0.010930
2023-01-10 11:00:00    0.030110
2023-01-10 12:00:00    0.029779
2023-01-10 13:00:00    0.040317
2023-01-10 14:00:00    0.056408
2023-01-10 15:00:00    0.096224
2023-01-10 16:00:00    0.013925
2023-01-10 17:00:00    0.012239
2023-01-10 18:00:00    0.017667
2023-01-10 19:00:00    0.233135
2023-01-10 20:00:00    0.070133
2023-01-10 21:00:00    0.002891
2023-01-10 22:00:00    0.002891
2023-01-10 23:00:00    0.002891
dtype: float64

In [9]:
# creating usage df
usage_df = prep.pipeline_usage(prep.input, usage_pipe_params)
usage_df

,activity,sensor.shellyplug_s_4022d88961b4_power_usage,sensor.shellyplug_s_4022d88984b8_power_usage,periods_since_last_activity,periods_since_last_sensor.shellyplug_s_4022d88961b4_power_usage,periods_since_last_sensor.shellyplug_s_4022d88984b8_power_usage,hour,activity_lag_1,activity_lag_2,activity_lag_3,...,sensor.shellyplug_s_4022d88984b8_power_usage_lag_1,sensor.shellyplug_s_4022d88984b8_power_usage_lag_2,sensor.shellyplug_s_4022d88984b8_power_usage_lag_3,active_last_2_days,day_name_Monday,day_name_Saturday,day_name_Sunday,day_name_Thursday,day_name_Tuesday,day_name_Wednesday
last_updated,,,,,,,,,,,,,,,,,,,,,
2022-12-25,1,1,1,NaN,NaN,NaN,0,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,1,0,0,0
2022-12-26,1,1,1,1.0,1.0,1.0,0,1.0,NaN,NaN,...,1.0,NaN,NaN,1,1,0,0,0,0,0
2022-12-27,1,1,1,1.0,1.0,1.0,0,1.0,1.0,NaN,...,1.0,1.0,NaN,1,0,0,0,0,1,0
2022-12-28,0,0,0,1.0,1.0,1.0,0,1.0,1.0,1.0,...,1.0,1.0,1.0,1,0,0,0,0,0,1
2022-12-29,1,1,0,2.0,2.0,2.0,0,0.0,1.0,1.0,...,0.0,1.0,1.0,1,0,0,0,1,0,0
2022-12-30,1,1,1,1.0,1.0,3.0,0,1.0,0.0,1.0,...,0.0,0.0,1.0,1,0,0,0,0,0,0
2022-12-31,1,0,1,1.0,1.0,1.0,0,1.0,1.0,0.0,...,1.0,0.0,0.0,1,0,1,0,0,0,0
2023-01-01,1,0,1,1.0,2.0,1.0,0,1.0,1.0,1.0,...,1.0,1.0,0.0,1,0,0,1,0,0,0
2023-01-02,1,1,0,1.0,3.0,1.0,0,1.0,1.0,1.0,...,1.0,1.0,1.0,1,1,0,0,0,0,0


In [12]:
usage = Usage_Agent(usage_df, device)
date = '2023-01-09'
train_start = ''
usage_prob, X_train_usage, X_test_usage, model_usage = usage.pipeline_xai(usage_df, date, 'random forest', train_start)

D:\Master BWL HU\3. Semester\Seminar Information Systems\Seminar-Information-Systems-main\agents.py:438: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return RandomForestClassifier(max_depth=max_depth, n_estimators=n_estimators, max_features=max_features, n_jobs=-1).fit(X, y)
C:\Users\chris\anaconda3\envs\recomm\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [13]:
usage_prob

array([0.51215238])

## **2.  Constructing the Load Agent**

### **2.1 Initialize Agent**

First we define the **Load Agent class**. It takes as input the data generated by the prep.pipeline_usage function computed above.

In [14]:
class Explainability_Agent:
    def __init__(self, model_activity, X_train_activity, X_test_activity, best_hour, model_usage,
               X_train_usage, X_test_usage, model_type):
        self.model_activity = model_activity
        self.model_type = model_type
        self.X_train_activity = X_train_activity
        self.X_test_activity = X_test_activity
        self.best_hour = best_hour
        self.model_usage = model_usage
        self.X_train_usage = X_train_usage
        self.X_test_usage = X_test_usage

### 2.2 Feature Importance


In [17]:
import shap as shap

In [18]:
def feature_importance(self):
    if self.model_type == "logit":
        X_train_summary = shap.sample(self.X_train_activity, 100)
        self.explainer_activity = shap.KernelExplainer(self.model_activity.predict_proba, X_train_summary)

    elif self.model_type == "ada":
        X_train_summary = shap.sample(self.X_train_activity, 100)
        self.explainer_activity = shap.KernelExplainer(self.model_activity.predict_proba, X_train_summary)

    elif self.model_type == "knn":
        X_train_summary = shap.sample(self.X_train_activity, 100)
        self.explainer_activity = shap.KernelExplainer(self.model_activity.predict_proba, X_train_summary)

    elif self.model_type == "random forest":

        self.explainer_activity = shap.TreeExplainer(self.model_activity, self.X_train_activity)

    elif self.model_type == "xgboost":
        self.explainer_activity = shap.TreeExplainer(self.model_activity, self.X_train_activity, model_output='predict_proba')
    else:
        raise InputError("Unknown model type.")


    self.shap_values = self.explainer_activity.shap_values(
        self.X_test_activity.iloc[self.best_hour, :])

    feature_names_activity = list(self.X_train_activity.columns.values)

    vals_activity = self.shap_values[1]

    feature_importance_activity = pd.DataFrame(list(zip(feature_names_activity, vals_activity)),
                                               columns=['col_name', 'feature_importance_vals'])
    feature_importance_activity.sort_values(by=['feature_importance_vals'], ascending=False, inplace=True)

    # usage
    if self.model_type == "logit":
        X_train_summary = shap.sample(self.X_train_usage, 100)
        self.explainer_usage = shap.KernelExplainer(self.model_usage.predict_proba, X_train_summary)


    elif self.model_type == "ada":
        X_train_summary = shap.sample(self.X_train_usage, 100)
        self.explainer_usage = shap.KernelExplainer(self.model_usage.predict_proba, X_train_summary)

    elif self.model_type == "knn":
        X_train_summary = shap.sample(self.X_train_usage, 100)
        self.explainer_usage = shap.KernelExplainer(self.model_usage.predict_proba, X_train_summary)

    elif self.model_type == "random forest":

        self.explainer_usage = shap.TreeExplainer(self.model_usage, self.X_train_usage)

    elif self.model_type == "xgboost":
        self.explainer_usage = shap.TreeExplainer(self.model_usage, self.X_train_usage, model_output='predict_proba')
    else:
        raise InputError("Unknown model type.")


    self.shap_values_usage = self.explainer_usage.shap_values(
        self.X_test_usage)

    feature_names_usage = list(self.X_train_usage.columns.values)

    vals = self.shap_values_usage[1]

    feature_importance_usage = pd.DataFrame(list(zip(feature_names_usage, vals)),
                                            columns=['col_name', 'feature_importance_vals'])
    feature_importance_usage.sort_values(by=['feature_importance_vals'], ascending=False, inplace=True)

    return feature_importance_activity, feature_importance_usage, self.explainer_activity, self.explainer_usage, self.shap_values, self.shap_values_usage, self.X_test_activity, self.X_test_usage

# add to Explainability agent
setattr(Explainability_Agent, 'feature_importance', feature_importance)
del feature_importance 


In [28]:
best_hour = 19
Explainability_Agent_i = Explainability_Agent(model_activity, X_train_activity, X_test_activity, best_hour, model_usage,
               X_train_usage, X_test_usage, model_type) 
feature_importance_activity, feature_importance_usage, explainer_activity, explainer_usage, shap_values, shap_values_usage, X_test_activity, X_test_usage = Explainability_Agent_i.feature_importance()
feature_importance_activity

,col_name,feature_importance_vals
0,hour,0.196181
7,day_name_Thursday,0.040027
5,day_name_Saturday,0.028307
9,day_name_Wednesday,0.021947
3,activity_lag_72,0.016128
1,activity_lag_24,0.014109
4,day_name_Monday,0.012265
6,day_name_Sunday,0.009619
2,activity_lag_48,0.005575
8,day_name_Tuesday,-0.190192


### 2.3 Explaination from feature importance for activity agent


In [36]:
weather_hourly = pd.read_pickle('D:/Master BWL HU/3. Semester/Seminar Information Systems/Seminar-Information-Systems-main//export/weather_unscaled_hourly.pkl')

In [37]:
weather_hourly

,Unix,Aggregate,Toaster,Fridge-Freezer,Freezer,Tumble Dryer,Dishwasher,Washing Machine,Television,Microwave,Kettle,Issues,dwpt,rhum,temp,wdir,wspd
Time,,,,,,,,,,,,,,,,,
2013-09-25 19:00:00,1.380138e+09,410.766578,0.000000,28.384615,36.310345,0.0,0.0,0.000000,144.262599,0.000000,0.000000,0.000000,16.0,14.0,88.0,50.0,9.4
2013-09-25 20:00:00,1.380141e+09,417.421053,0.000000,49.443124,44.348048,0.0,0.0,0.000000,143.928693,0.000000,0.000000,0.000000,16.0,12.9,82.0,40.0,7.6
2013-09-25 21:00:00,1.380145e+09,508.165821,0.000000,31.849408,36.592217,0.0,0.0,0.000000,67.302876,0.000000,0.000000,0.000000,15.0,13.0,88.0,30.0,13.0
2013-09-25 22:00:00,1.380148e+09,264.194492,0.000000,33.306368,35.734940,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,15.0,13.0,88.0,70.0,18.4
2013-09-25 23:00:00,1.380152e+09,296.035836,0.000000,99.110922,36.262799,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,13.0,12.1,94.0,60.0,20.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-06-02 06:00:00,1.433227e+09,289.996047,0.000000,45.592885,46.397233,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,12.0,11.1,94.0,240.0,22.3
2015-06-02 07:00:00,1.433230e+09,407.575581,0.000000,0.207364,35.753876,0.0,0.0,0.000000,3.546512,0.000000,136.724806,0.003876,13.0,12.1,94.0,230.0,25.9
2015-06-02 08:00:00,1.433234e+09,731.157390,41.857965,77.969290,37.124760,0.0,0.0,0.000000,143.180422,0.846449,101.712092,0.003839,14.0,12.0,88.0,220.0,29.5


In [38]:
def explanation_from_feature_importance_activity(self, feature_importance_activity, date, best_hour, diagnostics=False, weather_sel = False):
    self.feature_importance_activity = feature_importance_activity
    self.diagnostics = diagnostics

    sentence = 'We based the recommendation on your past activity and usage of the device. '

    #activity_lags:

    if self.X_test_activity['activity_lag_24'].iloc[self.best_hour] and self.X_test_activity['activity_lag_48'].iloc[self.best_hour] and self.X_test_activity['activity_lag_72'].iloc[self.best_hour] ==0:
        active_past = 'not '
    else:
        active_past = ''

    # input the activity lag with the strongest feature importance
    if feature_importance_activity.loc[feature_importance_activity['col_name']=='activity_lag_24','feature_importance_vals'].to_numpy()[0] >= 0 or \
            feature_importance_activity.loc[feature_importance_activity['col_name']=='activity_lag_48','feature_importance_vals'].to_numpy()[0] >= 0 or \
            feature_importance_activity.loc[feature_importance_activity['col_name']=='activity_lag_72','feature_importance_vals'].to_numpy()[0] >= 0:

            FI_lag = np.argmax([feature_importance_activity.loc[feature_importance_activity['col_name']=='activity_lag_24','feature_importance_vals'],
                               feature_importance_activity.loc[feature_importance_activity['col_name']=='activity_lag_48','feature_importance_vals'],
                               feature_importance_activity.loc[feature_importance_activity['col_name']=='activity_lag_72','feature_importance_vals']])

            if FI_lag == 0:
                activity_lag = 'day'
            elif FI_lag == 1:
                activity_lag = 'two days'
            elif FI_lag == 2:
                activity_lag = 'three days'
            else:
                activity_lag = 'three days'

            part1 = f"We believe you are active today since you were {active_past}active during the last {activity_lag}."
    else:
        part1 = ""
        
    if weather_sel:
        # weather:
        # need to rewrite that part afterwards, we need different weather data!

        # weather_hourly = pd.read_pickle('../export/weather_unscaled_hourly.pkl')


        d = {'features': ['dwpt', 'rhum', 'temp', 'wdir', 'wspd'],
             'labels': ['dewing point', 'relative humidity','temperature', 'wind direction', 'windspeed'],
             'feature_importances' : [feature_importance_activity.loc[feature_importance_activity[
                                                                'col_name'] == 'dwpt', 'feature_importance_vals'].to_numpy()[0],
                                  feature_importance_activity.loc[feature_importance_activity[
                                                                'col_name'] == 'rhum', 'feature_importance_vals'].to_numpy()[0],
                                  feature_importance_activity.loc[feature_importance_activity[
                                                                'col_name'] == 'temp', 'feature_importance_vals'].to_numpy()[0],
                                  feature_importance_activity.loc[feature_importance_activity[
                                                                      'col_name'] == 'wdir', 'feature_importance_vals'].to_numpy()[0],
                                  feature_importance_activity.loc[feature_importance_activity[
                                                                      'col_name'] == 'wspd', 'feature_importance_vals'].to_numpy()[0]],
             'feature_values' : [weather_hourly[date].iloc[best_hour, -5:].loc['dwpt'],
                                 weather_hourly[date].iloc[best_hour, -5:].loc['rhum'],
                                 weather_hourly[date].iloc[best_hour, -5:].loc['temp'],
                                 weather_hourly[date].iloc[best_hour, -5:].loc['wdir'],
                                 weather_hourly[date].iloc[best_hour, -5:].loc['wspd']
                                 ]

             }
        df = pd.DataFrame(data=d)

        sorted_df = df['feature_importances'].sort_values(ascending=False)
        if sorted_df.iloc[0] >= 0:
            weather1_ind = sorted_df.index[0]
            weather1 = df['labels'][weather1_ind]

            value1 = round(df['feature_values'][weather1_ind], 2)

            part2= f"The weather condition ({weather1}:{value1}) support that recommendation."

            if sorted_df.iloc[1] >= 0:

                weather2_ind = sorted_df.index[1]
                weather2 = df['labels'][weather2_ind]

                value2 = round(df['feature_values'][weather2_ind], 2)
                part2 = f"The weather conditions ({weather1}:{value1}, {weather2}:{value2}) support that recommendation."

        else:
            part2= ""
    else:
        part2 = ""

    # Time features
    # DAY
    day_names = ['day_name_Monday','day_name_Tuesday','day_name_Wednesday','day_name_Thursday','day_name_Saturday','day_name_Sunday']
    for day in day_names:
        if feature_importance_activity.loc[feature_importance_activity['col_name'] == day, 'feature_importance_vals'].to_numpy()[0] >= 0:
            part3 = "The weekday strenghtens that prediction."

            if feature_importance_activity.loc[
                feature_importance_activity['col_name'] == 'hour', 'feature_importance_vals'].to_numpy()[0] >= 0:
                part3 = "The weekday and hour strenghtens that prediction."

        else:
            part3 = ""
            if feature_importance_activity.loc[
                feature_importance_activity['col_name'] == 'hour', 'feature_importance_vals'].to_numpy()[0] >= 0:
                part3 = "The hour strenghtens that prediction."


    # final activity sentence
    sentence_activity = (str(part1) + str(part2)+ str(part3))

    explanation_sentence = sentence + sentence_activity

    return explanation_sentence

# add to Explainability agent
setattr(Explainability_Agent, 'explanation_from_feature_importance_activity', explanation_from_feature_importance_activity)
del explanation_from_feature_importance_activity 


In [48]:
best_hour = 19
Explainability_Agent_i = Explainability_Agent(model_activity, X_train_activity, X_test_activity, best_hour, model_usage,
               X_train_usage, X_test_usage, model_type) 
sentence = Explainability_Agent_i.explanation_from_feature_importance_activity(feature_importance_activity, date, best_hour, diagnostics=False, weather_sel = False)
sentence

'We based the recommendation on your past activity and usage of the device. We believe you are active today since you were active during the last three days.The weekday and hour strenghtens that prediction.'

### 2.4 Explaination from feature importance for usage agent


In [41]:
def explanation_from_feature_importance_usage(self, feature_importance_usage, date, diagnostics=False, weather_sel = False):

    self.feature_importance_usage= feature_importance_usage
    self.diagnostics = diagnostics

    if self.X_test_usage['active_last_2_days'] == 0:
        active_past = 'not'
    else:
        active_past = ''

    if feature_importance_usage.loc[0, 'feature_importance_vals'] >= 0 or feature_importance_usage.loc[1, 'feature_importance_vals'] >= 0:

        FI_lag = np.argmax([feature_importance_usage.loc[0, 'feature_importance_vals'],
                            feature_importance_usage.loc[1, 'feature_importance_vals']])

        if FI_lag == 0:
            device_usage = ""
            number_days = 'day'
        elif FI_lag == 1:
            device_usage = ""
            number_days = 'two days'
        else:
            device_usage = " not"
            number_days = 'two days'

        part1 = f" and have{device_usage} used the device in the last {number_days}"

    else:
        part1= ""

    if weather_sel:
        # weather:
        # need to rewrite that part afterwards, we need different weather data!
        weather_daily = pd.read_pickle('../export/weather_unscaled_daily.pkl')

        d = {'features': ['dwpt', 'rhum', 'temp', 'wdir', 'wspd'],
             'labels': ['dewing point', 'relative humidity','temperature', 'wind direction', 'windspeed'],
             'feature_importances' : [feature_importance_usage.loc[feature_importance_usage[
                                                                'col_name'] == 'dwpt', 'feature_importance_vals'].to_numpy()[0],
                                  feature_importance_usage.loc[feature_importance_usage[
                                                                'col_name'] == 'rhum', 'feature_importance_vals'].to_numpy()[0],
                                  feature_importance_usage.loc[feature_importance_usage[
                                                                'col_name'] == 'temp', 'feature_importance_vals'].to_numpy()[0],
                                  feature_importance_usage.loc[feature_importance_usage[
                                                                      'col_name'] == 'wdir', 'feature_importance_vals'].to_numpy()[0],
                                  feature_importance_usage.loc[feature_importance_usage[
                                                                      'col_name'] == 'wspd', 'feature_importance_vals'].to_numpy()[0]],
             'feature_values': [weather_daily.loc[date].loc['dwpt'],
                                weather_daily.loc[date].loc['rhum'],
                                weather_daily.loc[date].loc['temp'],
                                weather_daily.loc[date].loc['wdir'],
                                weather_daily.loc[date].loc['wspd']
             ]


             }
        df = pd.DataFrame(data=d)

        sorted_df = df['feature_importances'].sort_values(ascending=False)


        if sorted_df.iloc[0] >= 0:
            weather1_ind = sorted_df.index[0]
            weather1 = df['labels'][weather1_ind]

            value1 = round(df['feature_values'][weather1_ind], 2)

            part2= f"The weather condition ({weather1}:{value1}) support that recommendation."

            if sorted_df.iloc[1] >= 0:

                weather2_ind = sorted_df.index[1]
                weather2 = df['labels'][weather2_ind]

                value2 = round(df['feature_values'][weather2_ind], 2)
                part2 = f"The weather conditions ({weather1}:{value1}, {weather2}:{value2}) support that recommendation."

    else:
        part2 = ""

    sentence_usage = f"We believe you are likely to use the device in the near future since you " \
                     f"were {active_past}active during the last 2 days" + str(part1) + "." + str(part2)
    explanation_sentence = sentence_usage

    return explanation_sentence

# add to Explainability agent
setattr(Explainability_Agent, 'explanation_from_feature_importance_usage', explanation_from_feature_importance_usage)
del explanation_from_feature_importance_usage 


In [49]:
best_hour = 19
Explainability_Agent_i = Explainability_Agent(model_activity, X_train_activity, X_test_activity, best_hour, model_usage,
               X_train_usage, X_test_usage, model_type) 
sentence = Explainability_Agent_i.explanation_from_feature_importance_usage(feature_importance_usage, date, diagnostics=False, weather_sel = False)
sentence

'We believe you are likely to use the device in the near future since you were active during the last 2 days and have used the device in the last two days.'

# 3. Complete Explainability Agent

In [ ]:
# Explainability Agent
# ===============================================================================================
class Explainability_Agent:
    def __init__(self, model_activity, X_train_activity, X_test_activity, best_hour, model_usage,
               X_train_usage, X_test_usage, model_type):
        self.model_activity = model_activity
        self.model_type = model_type
        self.X_train_activity = X_train_activity
        self.X_test_activity = X_test_activity
        self.best_hour = best_hour
        self.model_usage = model_usage
        self.X_train_usage = X_train_usage
        self.X_test_usage = X_test_usage

    def feature_importance(self):
        if self.model_type == "logit":
            X_train_summary = shap.sample(self.X_train_activity, 100)
            self.explainer_activity = shap.KernelExplainer(self.model_activity.predict_proba, X_train_summary)

        elif self.model_type == "ada":
            X_train_summary = shap.sample(self.X_train_activity, 100)
            self.explainer_activity = shap.KernelExplainer(self.model_activity.predict_proba, X_train_summary)

        elif self.model_type == "knn":
            X_train_summary = shap.sample(self.X_train_activity, 100)
            self.explainer_activity = shap.KernelExplainer(self.model_activity.predict_proba, X_train_summary)

        elif self.model_type == "random forest":

            self.explainer_activity = shap.TreeExplainer(self.model_activity, self.X_train_activity)

        elif self.model_type == "xgboost":
            self.explainer_activity = shap.TreeExplainer(self.model_activity, self.X_train_activity, model_output='predict_proba')
        else:
            raise InputError("Unknown model type.")


        self.shap_values = self.explainer_activity.shap_values(
            self.X_test_activity.iloc[self.best_hour, :])

        feature_names_activity = list(self.X_train_activity.columns.values)

        vals_activity = self.shap_values[1]

        feature_importance_activity = pd.DataFrame(list(zip(feature_names_activity, vals_activity)),
                                                   columns=['col_name', 'feature_importance_vals'])
        feature_importance_activity.sort_values(by=['feature_importance_vals'], ascending=False, inplace=True)

        # usage
        if self.model_type == "logit":
            X_train_summary = shap.sample(self.X_train_usage, 100)
            self.explainer_usage = shap.KernelExplainer(self.model_usage.predict_proba, X_train_summary)


        elif self.model_type == "ada":
            X_train_summary = shap.sample(self.X_train_usage, 100)
            self.explainer_usage = shap.KernelExplainer(self.model_usage.predict_proba, X_train_summary)

        elif self.model_type == "knn":
            X_train_summary = shap.sample(self.X_train_usage, 100)
            self.explainer_usage = shap.KernelExplainer(self.model_usage.predict_proba, X_train_summary)

        elif self.model_type == "random forest":

            self.explainer_usage = shap.TreeExplainer(self.model_usage, self.X_train_usage)

        elif self.model_type == "xgboost":
            self.explainer_usage = shap.TreeExplainer(self.model_usage, self.X_train_usage, model_output='predict_proba')
        else:
            raise InputError("Unknown model type.")


        self.shap_values_usage = self.explainer_usage.shap_values(
            self.X_test_usage)

        feature_names_usage = list(self.X_train_usage.columns.values)

        vals = self.shap_values_usage[1]

        feature_importance_usage = pd.DataFrame(list(zip(feature_names_usage, vals)),
                                                columns=['col_name', 'feature_importance_vals'])
        feature_importance_usage.sort_values(by=['feature_importance_vals'], ascending=False, inplace=True)

        return feature_importance_activity, feature_importance_usage, self.explainer_activity, self.explainer_usage, self.shap_values, self.shap_values_usage, self.X_test_activity, self.X_test_usage

    def explanation_from_feature_importance_activity(self, feature_importance_activity, date, best_hour, diagnostics=False, weather_sel = False):
        self.feature_importance_activity = feature_importance_activity
        self.diagnostics = diagnostics

        sentence = 'We based the recommendation on your past activity and usage of the device. '

        #activity_lags:

        if self.X_test_activity['activity_lag_24'].iloc[self.best_hour] and self.X_test_activity['activity_lag_48'].iloc[self.best_hour] and self.X_test_activity['activity_lag_72'].iloc[self.best_hour] ==0:
            active_past = 'not '
        else:
            active_past = ''

        # input the activity lag with the strongest feature importance
        if feature_importance_activity.loc[feature_importance_activity['col_name']=='activity_lag_24','feature_importance_vals'].to_numpy()[0] >= 0 or \
                feature_importance_activity.loc[feature_importance_activity['col_name']=='activity_lag_48','feature_importance_vals'].to_numpy()[0] >= 0 or \
                feature_importance_activity.loc[feature_importance_activity['col_name']=='activity_lag_72','feature_importance_vals'].to_numpy()[0] >= 0:

                FI_lag = np.argmax([feature_importance_activity.loc[feature_importance_activity['col_name']=='activity_lag_24','feature_importance_vals'],
                                   feature_importance_activity.loc[feature_importance_activity['col_name']=='activity_lag_48','feature_importance_vals'],
                                   feature_importance_activity.loc[feature_importance_activity['col_name']=='activity_lag_72','feature_importance_vals']])

                if FI_lag == 0:
                    activity_lag = 'day'
                elif FI_lag == 1:
                    activity_lag = 'two days'
                elif FI_lag == 2:
                    activity_lag = 'three days'
                else:
                    activity_lag = 'three days'

                part1 = f"We believe you are active today since you were {active_past}active during the last {activity_lag}."
        else:
            part1 = ""

        if weather_sel:
            # weather:
            # need to rewrite that part afterwards, we need different weather data!

            # weather_hourly = pd.read_pickle('../export/weather_unscaled_hourly.pkl')


            d = {'features': ['dwpt', 'rhum', 'temp', 'wdir', 'wspd'],
                 'labels': ['dewing point', 'relative humidity','temperature', 'wind direction', 'windspeed'],
                 'feature_importances' : [feature_importance_activity.loc[feature_importance_activity[
                                                                    'col_name'] == 'dwpt', 'feature_importance_vals'].to_numpy()[0],
                                      feature_importance_activity.loc[feature_importance_activity[
                                                                    'col_name'] == 'rhum', 'feature_importance_vals'].to_numpy()[0],
                                      feature_importance_activity.loc[feature_importance_activity[
                                                                    'col_name'] == 'temp', 'feature_importance_vals'].to_numpy()[0],
                                      feature_importance_activity.loc[feature_importance_activity[
                                                                          'col_name'] == 'wdir', 'feature_importance_vals'].to_numpy()[0],
                                      feature_importance_activity.loc[feature_importance_activity[
                                                                          'col_name'] == 'wspd', 'feature_importance_vals'].to_numpy()[0]],
                 'feature_values' : [weather_hourly[date].iloc[best_hour, -5:].loc['dwpt'],
                                     weather_hourly[date].iloc[best_hour, -5:].loc['rhum'],
                                     weather_hourly[date].iloc[best_hour, -5:].loc['temp'],
                                     weather_hourly[date].iloc[best_hour, -5:].loc['wdir'],
                                     weather_hourly[date].iloc[best_hour, -5:].loc['wspd']
                                     ]

                 }
            df = pd.DataFrame(data=d)

            sorted_df = df['feature_importances'].sort_values(ascending=False)
            if sorted_df.iloc[0] >= 0:
                weather1_ind = sorted_df.index[0]
                weather1 = df['labels'][weather1_ind]

                value1 = round(df['feature_values'][weather1_ind], 2)

                part2= f"The weather condition ({weather1}:{value1}) support that recommendation."

                if sorted_df.iloc[1] >= 0:

                    weather2_ind = sorted_df.index[1]
                    weather2 = df['labels'][weather2_ind]

                    value2 = round(df['feature_values'][weather2_ind], 2)
                    part2 = f"The weather conditions ({weather1}:{value1}, {weather2}:{value2}) support that recommendation."

            else:
                part2= ""
        else:
            part2 = ""

        # Time features
        # DAY
        day_names = ['day_name_Monday','day_name_Tuesday','day_name_Wednesday','day_name_Thursday','day_name_Saturday','day_name_Sunday']
        for day in day_names:
            if feature_importance_activity.loc[feature_importance_activity['col_name'] == day, 'feature_importance_vals'].to_numpy()[0] >= 0:
                part3 = "The weekday strenghtens that prediction."

                if feature_importance_activity.loc[
                    feature_importance_activity['col_name'] == 'hour', 'feature_importance_vals'].to_numpy()[0] >= 0:
                    part3 = "The weekday and hour strenghtens that prediction."

            else:
                part3 = ""
                if feature_importance_activity.loc[
                    feature_importance_activity['col_name'] == 'hour', 'feature_importance_vals'].to_numpy()[0] >= 0:
                    part3 = "The hour strenghtens that prediction."


        # final activity sentence
        sentence_activity = (str(part1) + str(part2)+ str(part3))

        explanation_sentence = sentence + sentence_activity

        return explanation_sentence

    def explanation_from_feature_importance_usage(self, feature_importance_usage, date, diagnostics=False, weather_sel = False):

        self.feature_importance_usage= feature_importance_usage
        self.diagnostics = diagnostics

        if self.X_test_usage['active_last_2_days'] == 0:
            active_past = 'not'
        else:
            active_past = ''

        if feature_importance_usage.loc[0, 'feature_importance_vals'] >= 0 or feature_importance_usage.loc[1, 'feature_importance_vals'] >= 0:

            FI_lag = np.argmax([feature_importance_usage.loc[0, 'feature_importance_vals'],
                                feature_importance_usage.loc[1, 'feature_importance_vals']])

            if FI_lag == 0:
                device_usage = ""
                number_days = 'day'
            elif FI_lag == 1:
                device_usage = ""
                number_days = 'two days'
            else:
                device_usage = " not"
                number_days = 'two days'

            part1 = f" and have{device_usage} used the device in the last {number_days}"

        else:
            part1= ""

        if weather_sel:
            # weather:
            # need to rewrite that part afterwards, we need different weather data!
            weather_daily = pd.read_pickle('../export/weather_unscaled_daily.pkl')

            d = {'features': ['dwpt', 'rhum', 'temp', 'wdir', 'wspd'],
                 'labels': ['dewing point', 'relative humidity','temperature', 'wind direction', 'windspeed'],
                 'feature_importances' : [feature_importance_usage.loc[feature_importance_usage[
                                                                    'col_name'] == 'dwpt', 'feature_importance_vals'].to_numpy()[0],
                                      feature_importance_usage.loc[feature_importance_usage[
                                                                    'col_name'] == 'rhum', 'feature_importance_vals'].to_numpy()[0],
                                      feature_importance_usage.loc[feature_importance_usage[
                                                                    'col_name'] == 'temp', 'feature_importance_vals'].to_numpy()[0],
                                      feature_importance_usage.loc[feature_importance_usage[
                                                                          'col_name'] == 'wdir', 'feature_importance_vals'].to_numpy()[0],
                                      feature_importance_usage.loc[feature_importance_usage[
                                                                          'col_name'] == 'wspd', 'feature_importance_vals'].to_numpy()[0]],
                 'feature_values': [weather_daily.loc[date].loc['dwpt'],
                                    weather_daily.loc[date].loc['rhum'],
                                    weather_daily.loc[date].loc['temp'],
                                    weather_daily.loc[date].loc['wdir'],
                                    weather_daily.loc[date].loc['wspd']
                 ]


                 }
            df = pd.DataFrame(data=d)

            sorted_df = df['feature_importances'].sort_values(ascending=False)


            if sorted_df.iloc[0] >= 0:
                weather1_ind = sorted_df.index[0]
                weather1 = df['labels'][weather1_ind]

                value1 = round(df['feature_values'][weather1_ind], 2)

                part2= f"The weather condition ({weather1}:{value1}) support that recommendation."

                if sorted_df.iloc[1] >= 0:

                    weather2_ind = sorted_df.index[1]
                    weather2 = df['labels'][weather2_ind]

                    value2 = round(df['feature_values'][weather2_ind], 2)
                    part2 = f"The weather conditions ({weather1}:{value1}, {weather2}:{value2}) support that recommendation."

        else:
            part2 = ""

        sentence_usage = f"We believe you are likely to use the device in the near future since you " \
                         f"were {active_past}active during the last 2 days" + str(part1) + "." + str(part2)
        explanation_sentence = sentence_usage

        return explanation_sentence

